# Train model with different tools

## load data

In [3]:
import pandas
import numpy as np
import matplotlib.pyplot as plt

In [12]:
hd_ttH=pandas.read_hdf('../../Files/skimmed/ttH.h5', 'df')
hd_ttW=pandas.read_hdf('../../Files/skimmed/ttW.h5', 'df')

In [13]:
hd_ttH.head()
hd_ttH['DRll01']

entry
0        2.833694
1        2.171907
2        2.189412
3        2.389086
4        0.472404
5        0.717594
6        3.316627
7        2.160175
8        2.056099
9        0.709195
10       1.692138
11       3.211834
12       1.691607
13       1.851815
14       0.306701
15       0.795250
16       1.963612
17       0.613543
18       2.307259
19       0.852842
20       1.176668
21       2.964810
22       1.904576
23       2.985362
24       0.826391
25       1.260774
26       2.499718
27       2.962825
28       2.518363
29       3.475383
           ...   
69872    1.342597
69873    2.668161
69874    2.775584
69875    3.189132
69876    0.963059
69877    2.338224
69878    0.943902
69879    1.573434
69880    0.613034
69881    3.002996
69882    3.243807
69883    0.566110
69884    1.860721
69885    3.221114
69886    1.047707
69887    3.153875
69888    1.170774
69889    0.506986
69890    2.351830
69891    3.006831
69892    3.134802
69893    3.031784
69894    1.774774
69895    2.093268
6989

 * X = signal + background data;
 * object y - hold the class labels.

The correspondence between `X` and `y` is given by the index

In [14]:
X = np.concatenate((hd_ttH,hd_ttW)) # training data                                                                                                                               
y = np.concatenate((np.ones(hd_ttH.shape[0]),np.zeros(hd_ttW.shape[0]))) # class lables                                                                                           
